In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from time import sleep
from bs4 import BeautifulSoup
import csv
from Khong_dau import no_accent_vietnamese

In [2]:
# Mở trình duyệt Chrome
driver = webdriver.Chrome()
url = 'https://www.linkedin.com/home'
# Mở url
driver.get(url)

#Lưu username và password 
credential = open('login_credential.txt') 
lines = credential.readlines()
username = lines[0]
password = lines[1]

#Nhập username
email_field = driver.find_element(By.ID,'session_key')
email_field.send_keys(username)
sleep(2)

#Nhập password
password_field = driver.find_element(By.NAME, 'session_password')
password_field.send_keys(password)
sleep(2)

# Ấn nút login
login_field = driver.find_element(By.XPATH, '//*[@id="main-content"]/section[1]/div/div/form/div[2]/button')
login_field.click()
sleep(2)


In [3]:
# # Nhập từ khóa muốn tìm kiếm
# search_field = driver.find_element(By.XPATH, '//*[@id="global-nav-typeahead"]/input')
# search_field.send_keys('Data Engineer')
# search_field.send_keys(Keys.ENTER)
# sleep(10)

In [4]:
# # Ấn nút people
# people_btn = driver.find_element(By.XPATH, '//*[@id="search-reusables__filters-bar"]/ul/li[3]/button')
# people_btn.click()
# sleep(2)

In [5]:
all_profiles_url_all_pages = []
for page in range(1,11):
    sleep(5)
    url = 'https://www.linkedin.com/search/results/people/?keywords=Data%20Engineer&origin=GLOBAL_SEARCH_HEADER&page=' + str(page)
    driver.get(url)
    page_source = BeautifulSoup(driver.page_source)
    profiles = page_source.find_all('a', attrs={"class": 'app-aware-link', "aria-hidden": 'true'})
    
    for profile in profiles:
        profile_id = profile.get('href')
        if profile_id not in all_profiles_url_all_pages and profile_id.endswith('Data%20Engineer') == 0:
            all_profiles_url_all_pages.append(profile_id)
            
print(len(all_profiles_url_all_pages))
print(all_profiles_url_all_pages)


82
['https://www.linkedin.com/in/chudang10031998?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACnNp6YBaFNtPJF7IwWseTp7IBos8OZDH8c', 'https://www.linkedin.com/in/tranchuyen282?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADVRqOcBj5IjJcsRfW8CRldbVNN848J1WUA', 'https://www.linkedin.com/in/huytv99?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACvFjjABUqQETeE8CBx41e6Phm6CESlK-kY', 'https://www.linkedin.com/in/raisinbl?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACSNx2kB_7XvmMRNgR0QfpASbfhgEF25snE', 'https://www.linkedin.com/in/sophie-nguyenthuthuy?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAC8DB3gBwwnd6zQGGvukMN6kayKRQDdCNGM', 'https://www.linkedin.com/in/anh-minh-le-62368113a?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACHqOjsBp7SKSsTe9_uFgFhLBjgGXTdzwFY', 'https://www.linkedin.com/in/chiennguyenchu?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAD7LP3gBFhC1HSGMtGiAF6NB2Hh0mw43rtI', 'https://www.linkedin.com/in/minh-nguyen-nguyen-b2b9b1205?miniProfileUrn=urn%3Ali%3Afs_miniPr

In [9]:
with open('output.csv', 'w',  newline='') as file_output:
    headers = ['Name', 'Job Title', 'Location', 'Company', 'Education',"URL"]
    writer = csv.DictWriter(file_output, delimiter=',',
                            lineterminator='\n', fieldnames=headers)
    writer.writeheader()
    for url_profile in all_profiles_url_all_pages[:]:
        driver.get(url_profile)
        sleep(5)
        page_source=BeautifulSoup(driver.page_source)
        try:
                #Lấy tên
                name_field = page_source.find('div', class_='pv-text-details__title')
                name_loc = name_field.find_all('h1', class_='text-heading-xlarge inline t-24 v-align-middle break-words')
                name = no_accent_vietnamese(name_loc[0].get_text().strip()).encode('utf8')
                print(name)

                #Lấy chức vụ
                job_field = page_source.find('div', class_='text-body-medium break-words')
                job = no_accent_vietnamese(job_field.get_text()).strip().encode('utf8')
                print(job)
                
                #Lấy địa chỉ
                location_field = page_source.find('div', class_='pv-text-details__left-panel mt2')
                location = location_field.find('span', class_='text-body-small inline t-black--light break-words').get_text()
                location = no_accent_vietnamese(location).strip().encode('utf8')
                print(location)
                
                company='empty'
                education='empty'
                # Lấy thẻ ul chứa thông tin công ty và trường đại học
                temp = page_source.find('ul', class_='pv-text-details__right-panel')
                
                company_education_field = temp.find_all(
                        'div', class_='inline-show-more-text inline-show-more-text--is-collapsed inline-show-more-text--is-collapsed-with-line-clamp inline')
                
                if(company_education_field != None):
                        if len(company_education_field)==2:
                                company = company_education_field[0].get_text().strip()
                                education = company_education_field[1].get_text().strip()
                        elif len(company_education_field)==1:
                                company = company_education_field[0].get_text().strip()
                company = no_accent_vietnamese(company).encode('utf8')
                education = no_accent_vietnamese(education).encode('utf8')
                #Lấy công ty
                print(company)
                #Láy trường đại học
                print(education)
                print('--------------------')
                writer.writerow({headers[0]: name, headers[1]: job, headers[2]: location,\
                                 headers[3]: company, headers[4]: education,headers[5]:url_profile.encode('utf8')})
        except:
                print(NameError)
    


b'Chu Van Dang'
b'Data Engineer at One Mount'
b'Hanoi Capital Region'
b'One Mount'
b'Posts and Telecommunications Institute of Technology'
--------------------
b'Tran Chuyen'
b'Data Engineer at MedOn JSC'
b'Hanoi Capital Region'
b'MedOn JSC'
b'Posts and Telecommunications Institute of Technology'
--------------------
b'Huy Tran'
b'Senior Data Engineer | Cloud DevOps | Software Engineer'
b'Hanoi, Hanoi, Vietnam'
b'FPT Software'
b'empty'
--------------------
b'Hung Duy'
b'Data Engineer'
b'Hanoi Capital Region'
b'Viettel Software'
b'Hanoi University of Science and Technology'
--------------------
b'Sophie N.'
b'Data Engineer at TalentFirst.ai'
b'Hoang Mai District, Hanoi, Vietnam'
b'TalentFirst.ai'
b'University of Colorado Boulder'
--------------------
b'Anh Minh Le'
b'Data Engineer'
b'Hanoi, Hanoi, Vietnam'
b'Viettel Telecom'
b'University of Wollongong'
--------------------
b'Chien Nguyen'
b'Middle Data Engineer'
b'Hanoi Capital Region'
b'FPT Software'
b'VNU University of Engineering and

In [7]:
print(len(all_profiles_url_all_pages))
print("Nguyễn")

82
Nguyễn
